In [104]:
import asyncio
from pathlib import Path  # Используем pathlib для удобной и кросс-платформенной работы с путями
import uuid
from typing import Optional, Coroutine, Any

from sensory_data_client import DataClient, get_settings, DataClientConfig, PostgresConfig, MinioConfig, create_data_client
from sensory_data_client.models.document import DocumentCreate, DocumentMetadata
from pydantic import BaseModel, Field
cfg = DataClientConfig(postgres = PostgresConfig(user = "postgres",
                                                           password = "postgres",
                                                           host = "10.10.0.70",
                                                           port=5422), 
                                            minio = MinioConfig(endpoint="10.10.0.70:9008",  bucket = "documents"))
client = create_data_client(cfg)
await client.check_connections()

endpoint='10.10.0.70:9008' accesskey='minioadmin' secretkey='minioadmin' bucket='documents' secure=False


{'postgres': 'ok', 'minio': 'ok'}

In [3]:

lines  = await client.list_doclines()
objects = await client.list_stor(prefix="pdf/")

In [4]:
objects

[]

In [173]:
file_path_str = r"X:\Telegram Desktop\Bain_Technology Report_2024.pdf"
file_path = Path(file_path_str)

# Проверка, что файл существует, перед тем как его читать
if not file_path.is_file():
    print(f"Ошибка: Файл не найден по пути {file_path}")

print(f"Подготовка к загрузке файла: {file_path}")

# Шаг 2: Читаем файл в байты
# .read_bytes() - простой способ прочитать весь файл в память
try:
    file_content: bytes = file_path.read_bytes()
except Exception as e:
    print(f"Не удалось прочитать файл: {e}")
    

# Шаг 3: Создаем объект метаданных DocumentCreate
# Извлекаем информацию из пути и добавляем остальную
document_meta = DocumentCreate(
    user_document_id=str(uuid.uuid4()), # Генерируем уникальный ID для этого документа
    name=file_path.name,                # 'X5Group x SensoryLAB 07.02.2024.pptx.pdf'
    owner="user-alpha-007",             # ID пользователя, который загружает файл
    access_group="x5-auditors",         # Опциональная группа доступа
    metadata=DocumentMetadata()
)


Подготовка к загрузке файла: X:\Telegram Desktop\Bain_Technology Report_2024.pdf


In [175]:

# Шаг 4: Вызываем асинхронный метод с `await`
try:
    # Ваш метод `upload_file` скорее всего захочет получить чистое имя файла,
    # а не весь путь. file_path.name идеально для этого подходит.
    result_document = await client.upload_file(
        file_name=file_path.name,
        content=file_content,
        meta=document_meta
    )
    
    print("\n✅ Успешно загружен документ!")
    print(f"   ID в базе: {result_document.id}")
    print(f"   Путь в хранилище: {result_document}")

except Exception as e:
    print(f"❌ Произошла ошибка во время загрузки: {e}")



✅ Успешно загружен документ!
   ID в базе: 89abe34d-2838-4030-8373-6f6923a7c809
   Путь в хранилище: user_document_id='ca8c0248-4557-4e50-b8e9-71674759e8e4' name='Bain_Technology Report_2024.pdf' owner='user-alpha-007' access_group='x5-auditors' metadata=DocumentMetadata(processing_status=None, image_object_paths=None, extra=None) id=UUID('89abe34d-2838-4030-8373-6f6923a7c809') created=datetime.datetime(2025, 8, 12, 14, 19, 38, 797974, tzinfo=datetime.timezone.utc) edited=datetime.datetime(2025, 8, 12, 14, 19, 38, 797974, tzinfo=datetime.timezone.utc) content_hash='9362e5fac0f6b1ad11ba321a30526dabdb3b01e02649d4c6285c364f34fe061c' object_path='pdf/89abe34d2838403083736f6923a7c809/raw/Bain_Technology Report_2024.pdf' extension='pdf'
